In [0]:
!pip install tmdbsimple
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32917 sha256=d91124f348b4e1458d39f3144bf1c20ff02dde6ac1843bc521b0ed476994aeda
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [0]:
import pandas as pd

df = pd.read_pickle("/content/drive/My Drive/promise_last.pckl")

In [0]:
len(df)

42978

In [0]:
from gensim import models
# model2 = models.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True) 
model2 = models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
tokenizer = RegexpTokenizer(r'\w+')

en_stop = get_stop_words('en')


In [0]:
len(en_stop)

174

In [0]:
import numpy as np

movie_mean_wordvec=np.zeros((len(df),300))
movie_mean_wordvec.shape


(42978, 300)

In [0]:
genres=[]
rows_to_delete=[]
for i in range(len(df)):
    movie_genres=df.genre_ids[i]
    genres.append(movie_genres)
    overview=df.overview[i]
    tokens = tokenizer.tokenize(overview)
    stopped_tokens = [k for k in tokens if not k in en_stop]
    count_in_vocab=0
    s=0
    if len(stopped_tokens)==0:
        rows_to_delete.append(i)
        genres.pop(-1)
#         print(overview)
#         print("sample ",i,"had no nonstops")
    else:
        for tok in stopped_tokens:
            if tok.lower() in model2.vocab:
                count_in_vocab+=1
                s+=model2[tok.lower()]
        if count_in_vocab!=0:
            movie_mean_wordvec[i]=s/float(count_in_vocab)
        else:
            rows_to_delete.append(i)
            genres.pop(-1)
#             print(overview)
#             print("sample ",i,"had no word2vec")

In [0]:
rows_to_delete
#df.title[11043]
#df.overview[11043]


[]

In [0]:
df = df[df.title!="Bright 2"]

In [0]:
df = df[df.title!="Fuggy Fuggy"]

In [0]:
len(df)

42989

In [0]:
df = df.reset_index()

In [0]:
df

,title,overview,genre_ids,text_vec,poster_path,image_vec
0,Travis and the Nitro Circus,"In January 2003, NC1 exploded on the scene and...","[28, 12]",[ 0.01052348 0.02272458 0.00019497 0.060398...,/content/drive/My Drive/storage1/Travis_and_th...,[[[[ 0. 0. 0. 0. ...
1,Robot Chicken: Star Wars,"Fans of Adult Swim's ""Robot Chicken"" and the S...","[16, 35, 878]",[ 0.0239022 0.06350569 -0.00177904 0.101187...,/content/drive/My Drive/storage1/Robot_Chicken...,[[[[ 0. 0. 0. 0. ...
2,Sense8: Creating the World,Go behind the scenes and around the world with...,"[99, 878]",[ 0.00421143 0.0841039 0.01454163 -0.017206...,/content/drive/My Drive/storage1/Sense8:_Creat...,[[[[ 0. 0. 0. 0. ...
3,North of Nome,John Raglan is a seal hunter being hounded by ...,"[80, 12]",[ 0.01378632 0.05131454 -0.06986656 -0.046843...,/content/drive/My Drive/storage1/North_of_Nome...,[[[[ 0. 0. 0. 0. ...
4,SPIT: The Story of a Caveman and a Chicken,A quirky caveman sets out on a journey to find...,"[35, 12]",[ 0.0442276 0.05976868 -0.03481674 0.121612...,/content/drive/My Drive/storage1/SPIT:_The_Sto...,[[[[ 0. 0. 0. 0. ...
...,...,...,...,...,...,...
42973,Get Lucky,Lucky is a small time criminal and he's happy ...,[28],[ 7.19434917e-02 5.22133149e-02 -1.28956549e-...,/content/drive/My Drive/storage3/Get_Lucky.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42974,Four Frightened People,Four passengers escape their bubonic plague-in...,"[12, 18, 53]",[ 0.08907948 0.14613475 -0.0722444 0.031435...,/content/drive/My Drive/storage3/Four_Frighten...,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42975,Iron Will,"When Will Stoneman's father dies, he is left a...",[12],[ 3.68289053e-02 8.44523087e-02 1.30716963e-...,/content/drive/My Drive/storage3/Iron_Will.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42976,Bran Nue Dae,In the Summer of 1965 a young man is filled wi...,"[35, 18, 10749]",[ 0.08937211 0.0896907 0.00616825 -0.002673...,/content/drive/My Drive/storage3/Bran_Nue_Dae.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...


In [0]:
df = df.drop("Unnamed: 0",axis=1)
df = df.drop("index",axis=1)


In [0]:
df

,title,overview,genre_ids
0,Travis and the Nitro Circus,"In January 2003, NC1 exploded on the scene and...","[28, 12]"
1,Around India with a Movie Camera,Award-winning filmmaker Sandhya Suri (I for In...,[99]
2,Robot Chicken: Star Wars,"Fans of Adult Swim's ""Robot Chicken"" and the S...","[16, 35, 878]"
3,Sense8: Creating the World,Go behind the scenes and around the world with...,"[99, 878]"
4,Separated At Birth,Lucy Pierce grew up believing she had a normal...,[53]
...,...,...,...
42984,Thank Your Lucky Stars,An Eddie Cantor look-alike organizes an all-st...,"[35, 10402]"
42985,Morbius,Biochemist Michael Morbius tries to cure himse...,"[28, 27, 878]"
42986,King Dave,"The story of Dave, a self-proclaimed king. Tol...","[18, 53]"
42987,The Midnight Game,"After playing a pagan ritual on a dare, a grou...","[53, 27]"


In [0]:
mask2=[]
for row in range(len(movie_mean_wordvec)):
    if row in rows_to_delete:
        mask2.append(False)
    else:
        mask2.append(True)

In [0]:
X = movie_mean_wordvec[mask2]

In [0]:
X.shape

(42978, 300)

In [0]:
text_vec = []

In [0]:
for i in range(len(X)):
  text_vec.append(X[i])

In [0]:
df.drop("text_vec",axis=1)

,title,overview,genre_ids,poster_path,image_vec
0,Travis and the Nitro Circus,"In January 2003, NC1 exploded on the scene and...","[28, 12]",/content/drive/My Drive/storage1/Travis_and_th...,[[[[ 0. 0. 0. 0. ...
1,Robot Chicken: Star Wars,"Fans of Adult Swim's ""Robot Chicken"" and the S...","[16, 35, 878]",/content/drive/My Drive/storage1/Robot_Chicken...,[[[[ 0. 0. 0. 0. ...
2,Sense8: Creating the World,Go behind the scenes and around the world with...,"[99, 878]",/content/drive/My Drive/storage1/Sense8:_Creat...,[[[[ 0. 0. 0. 0. ...
3,North of Nome,John Raglan is a seal hunter being hounded by ...,"[80, 12]",/content/drive/My Drive/storage1/North_of_Nome...,[[[[ 0. 0. 0. 0. ...
4,SPIT: The Story of a Caveman and a Chicken,A quirky caveman sets out on a journey to find...,"[35, 12]",/content/drive/My Drive/storage1/SPIT:_The_Sto...,[[[[ 0. 0. 0. 0. ...
...,...,...,...,...,...
42973,Get Lucky,Lucky is a small time criminal and he's happy ...,[28],/content/drive/My Drive/storage3/Get_Lucky.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42974,Four Frightened People,Four passengers escape their bubonic plague-in...,"[12, 18, 53]",/content/drive/My Drive/storage3/Four_Frighten...,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42975,Iron Will,"When Will Stoneman's father dies, he is left a...",[12],/content/drive/My Drive/storage3/Iron_Will.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42976,Bran Nue Dae,In the Summer of 1965 a young man is filled wi...,"[35, 18, 10749]",/content/drive/My Drive/storage3/Bran_Nue_Dae.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...


In [0]:
df['text_vec'] = text_vec

In [0]:
df

,title,overview,genre_ids,text_vec,poster_path,image_vec
0,Travis and the Nitro Circus,"In January 2003, NC1 exploded on the scene and...","[28, 12]","[0.010523478500545025, 0.02272457629442215, 0....",/content/drive/My Drive/storage1/Travis_and_th...,[[[[ 0. 0. 0. 0. ...
1,Robot Chicken: Star Wars,"Fans of Adult Swim's ""Robot Chicken"" and the S...","[16, 35, 878]","[0.023902200162410736, 0.06350569427013397, -0...",/content/drive/My Drive/storage1/Robot_Chicken...,[[[[ 0. 0. 0. 0. ...
2,Sense8: Creating the World,Go behind the scenes and around the world with...,"[99, 878]","[0.00421142578125, 0.08410390466451645, 0.0145...",/content/drive/My Drive/storage1/Sense8:_Creat...,[[[[ 0. 0. 0. 0. ...
3,North of Nome,John Raglan is a seal hunter being hounded by ...,"[80, 12]","[0.01378631591796875, 0.051314543932676315, -0...",/content/drive/My Drive/storage1/North_of_Nome...,[[[[ 0. 0. 0. 0. ...
4,SPIT: The Story of a Caveman and a Chicken,A quirky caveman sets out on a journey to find...,"[35, 12]","[0.04422760009765625, 0.0597686767578125, -0.0...",/content/drive/My Drive/storage1/SPIT:_The_Sto...,[[[[ 0. 0. 0. 0. ...
...,...,...,...,...,...,...
42973,Get Lucky,Lucky is a small time criminal and he's happy ...,[28],"[0.0719434916973114, 0.052213314920663834, -0....",/content/drive/My Drive/storage3/Get_Lucky.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42974,Four Frightened People,Four passengers escape their bubonic plague-in...,"[12, 18, 53]","[0.08907948434352875, 0.14613474905490875, -0....",/content/drive/My Drive/storage3/Four_Frighten...,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42975,Iron Will,"When Will Stoneman's father dies, he is left a...",[12],"[0.0368289053440094, 0.0844523087143898, 0.013...",/content/drive/My Drive/storage3/Iron_Will.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42976,Bran Nue Dae,In the Summer of 1965 a young man is filled wi...,"[35, 18, 10749]","[0.08937211334705353, 0.08969070017337799, 0.0...",/content/drive/My Drive/storage3/Bran_Nue_Dae.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...


In [0]:
type(df.image_vec[0])

numpy.ndarray

In [0]:
df.to_pickle("pinky_promise_last.pckl")

In [0]:
dfx = pd.read_pickle("pinky_promise_last.pckl")

In [0]:
dfx

,title,overview,genre_ids,text_vec,poster_path,image_vec
0,Travis and the Nitro Circus,"In January 2003, NC1 exploded on the scene and...","[28, 12]","[0.010523478500545025, 0.02272457629442215, 0....",/content/drive/My Drive/storage1/Travis_and_th...,[[[[ 0. 0. 0. 0. ...
1,Robot Chicken: Star Wars,"Fans of Adult Swim's ""Robot Chicken"" and the S...","[16, 35, 878]","[0.023902200162410736, 0.06350569427013397, -0...",/content/drive/My Drive/storage1/Robot_Chicken...,[[[[ 0. 0. 0. 0. ...
2,Sense8: Creating the World,Go behind the scenes and around the world with...,"[99, 878]","[0.00421142578125, 0.08410390466451645, 0.0145...",/content/drive/My Drive/storage1/Sense8:_Creat...,[[[[ 0. 0. 0. 0. ...
3,North of Nome,John Raglan is a seal hunter being hounded by ...,"[80, 12]","[0.01378631591796875, 0.051314543932676315, -0...",/content/drive/My Drive/storage1/North_of_Nome...,[[[[ 0. 0. 0. 0. ...
4,SPIT: The Story of a Caveman and a Chicken,A quirky caveman sets out on a journey to find...,"[35, 12]","[0.04422760009765625, 0.0597686767578125, -0.0...",/content/drive/My Drive/storage1/SPIT:_The_Sto...,[[[[ 0. 0. 0. 0. ...
...,...,...,...,...,...,...
42973,Get Lucky,Lucky is a small time criminal and he's happy ...,[28],"[0.0719434916973114, 0.052213314920663834, -0....",/content/drive/My Drive/storage3/Get_Lucky.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42974,Four Frightened People,Four passengers escape their bubonic plague-in...,"[12, 18, 53]","[0.08907948434352875, 0.14613474905490875, -0....",/content/drive/My Drive/storage3/Four_Frighten...,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42975,Iron Will,"When Will Stoneman's father dies, he is left a...",[12],"[0.0368289053440094, 0.0844523087143898, 0.013...",/content/drive/My Drive/storage3/Iron_Will.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42976,Bran Nue Dae,In the Summer of 1965 a young man is filled wi...,"[35, 18, 10749]","[0.08937211334705353, 0.08969070017337799, 0.0...",/content/drive/My Drive/storage3/Bran_Nue_Dae.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...


In [0]:
type(df.image_vec[0])

numpy.ndarray